In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

In [55]:
data=pd.read_csv("tmdb_5000_movies.csv")
data=data[['id','genres','vote_average','vote_count','popularity', 'title','keywords','overview']]

In [3]:
tmp_m=data['vote_count'].quantile(0.89)
data['vote_count']>=tmp_m
tmp_data=data.copy()[data['vote_count']>=tmp_m]
tmp_data.shape

m=data['vote_count'].quantile(0.9)
data=data[data['vote_count']>=m]
data.shape

(481, 20)

In [4]:
c=data['vote_average'].mean()
c
m

1838.4000000000015

In [56]:
def weighted_rating(x, m=m, c=c):
    v=x['vote_count']
    R=x['vote_average']
    return (v/(v+m)*R)+(m/(m+v)*c)
data['score']=data.apply(weighted_rating, axis=1)

In [6]:
import ast

In [7]:
data['genres']=data['genres'].apply(ast.literal_eval)
data['keywords']=data['keywords'].apply(ast.literal_eval)

In [8]:
data['genres']=data['genres'].apply(lambda x: [d['name'] for d in x]).apply(lambda x: " ".join(x))
data['keywords']=data['keywords'].apply(lambda x: [d['name'] for d in x]).apply(lambda x: " ".join(x))

In [9]:
data.head()
data.to_csv("pre_tmdb_5000_movies.csv", index=False)

In [12]:
from sklearn.feature_extraction.text import CountVectorizer

In [13]:
counter_vector=CountVectorizer(ngram_range=(1,3))
c_vector_genres=counter_vector.fit_transform(data['genres'])

In [14]:
print(c_vector_genres.toarray())
print(c_vector_genres.shape)

[[1 1 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 [1 1 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
(481, 364)


In [15]:
counter_vector=CountVectorizer(ngram_range=(1,3))
counter_vector.fit(data['genres'])

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 3), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)

In [16]:
c_vector_genres.toarray().shape
print(data.iloc[0].genres)
print(c_vector_genres.toarray()[0])
print(data.iloc[1].genres)
print(c_vector_genres.toarray()[1])

Action Adventure Fantasy Science Fiction
[1 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0
 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Adventure Fantasy Action
[1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 

In [17]:
from sklearn.metrics.pairwise import cosine_similarity

In [18]:
c_vector_genres

<481x364 sparse matrix of type '<class 'numpy.int64'>'
	with 3238 stored elements in Compressed Sparse Row format>

In [20]:
cosine_similarity(c_vector_genres, c_vector_genres).shape
cosine_similarity(c_vector_genres, c_vector_genres)

array([[1.        , 0.47140452, 0.35355339, ..., 0.        , 0.        ,
        0.        ],
       [0.47140452, 1.        , 0.33333333, ..., 0.        , 0.        ,
        0.        ],
       [0.35355339, 0.33333333, 1.        , ..., 0.        , 0.23570226,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 1.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.23570226, ..., 0.        , 1.        ,
        0.57735027],
       [0.        , 0.        , 0.        , ..., 0.        , 0.57735027,
        1.        ]])

In [41]:
cosine_similarity(c_vector_genres, c_vector_genres)[0]
genres_c_sim=cosine_similarity(c_vector_genres, c_vector_genres).argsort()[:,::-1] # 각 영화별 코사인 유사도 값을 기준으로 내림차순 정렬

In [57]:
def get_recommend_movie_list(df, movie_title, top=30):
    target_movie_index=df[df['title']==movie_title].index.values
    sim_index=genres_c_sim[target_movie_index, :top].reshape(-1)
    sim_index=sim_index[sim_index!=target_movie_index]
    res=df.iloc[sim_index].sort_values('score', ascending=False)[:10]
    return res

get_recommend_movie_list(data, movie_title="The Dark Knight Rises")

,id,genres,vote_average,vote_count,popularity,title,keywords,overview,score
316,3131,"[{""id"": 18, ""name"": ""Drama""}, {""id"": 36, ""name...",7.1,1910,46.160048,Gangs of New York,"[{""id"": 657, ""name"": ""fire""}, {""id"": 1857, ""na...",It's 1863. America was born in the streets. Am...,7.032805
97,315011,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",6.5,143,9.476999,Shin Godzilla,"[{""id"": 1299, ""name"": ""monster""}, {""id"": 7671,...",From the mind behind Evangelion comes a hit la...,6.929579
83,79698,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",4.8,34,2.418535,The Lovers,[],The Lovers is an epic romance time travel adve...,6.923717
395,1581,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 10749, ""...",6.7,1225,44.967453,The Holiday,"[{""id"": 65, ""name"": ""holiday""}, {""id"": 212, ""n...","Two women, one (Cameron Diaz) from America and...",6.857827
479,77951,"[{""id"": 16, ""name"": ""Animation""}, {""id"": 10751...",5.2,133,10.088006,Walking With Dinosaurs,"[{""id"": 12616, ""name"": ""dinosaur""}, {""id"": 209...",Walking with Dinosaurs 3D is a film depicting ...,6.844054
293,16523,"[{""id"": 10751, ""name"": ""Family""}, {""id"": 14, ""...",6.4,572,31.586215,Where the Wild Things Are,"[{""id"": 170173, ""name"": ""children's book""}, {""...",Max imagines running away from his mom and sai...,6.829393
190,17578,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 16, ""...",6.7,2061,89.938296,The Adventures of Tintin,"[{""id"": 483, ""name"": ""riddle""}, {""id"": 1316, ""...","Intrepid young reporter, Tintin and his loyal ...",6.823990
312,22972,"[{""id"": 10752, ""name"": ""War""}, {""id"": 28, ""nam...",6.4,717,30.254021,Green Zone,"[{""id"": 41177, ""name"": ""weapon of mass destruc...",During the U.S.-led occupation of Baghdad in 2...,6.805028
311,11692,"[{""id"": 28, ""name"": ""Action""}, {""id"": 35, ""nam...",4.4,142,12.092241,The Adventures of Pluto Nash,"[{""id"": 305, ""name"": ""moon""}, {""id"": 585, ""nam...","The year is 2087, the setting is the moon. Plu...",6.779220
111,1858,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 878, ...",6.6,4040,25.468493,Transformers,"[{""id"": 2535, ""name"": ""destroy""}, {""id"": 4375,...","Young teenager, Sam Witwicky becomes involved ...",6.713522


In [27]:
x=np.array([20,10,30,13])
x.argsort()
np.sort(x)[::-1]
np.argsort(x)
np.argsort(-x)
x[np.argsort(x)]
x[np.argsort(-x)]

array([30, 20, 13, 10])

In [33]:
data=np.random.randn(4,3)
print(data)
print(data[:,0])
print(data[:,0].argsort())
print(data[:,0].argsort()[::-1])
print(data[data[:,0].argsort()[::-1]])

[[ 0.54511259  0.52543154 -1.04362138]
 [-0.701304    1.05404155 -2.01651487]
 [ 0.55062265 -1.35779168 -0.47050728]
 [ 0.74033337 -0.34245914 -0.38411407]]
[ 0.54511259 -0.701304    0.55062265  0.74033337]
[1 0 2 3]
[3 2 0 1]
[[ 0.74033337 -0.34245914 -0.38411407]
 [ 0.55062265 -1.35779168 -0.47050728]
 [ 0.54511259  0.52543154 -1.04362138]
 [-0.701304    1.05404155 -2.01651487]]


In [63]:
data=pd.read_csv("ratings_small.csv")
data=data.pivot_table('rating', index='userId', columns='movieId')
print(data)

movieId  1       2       3       4       5       6       7       8       \
userId                                                                    
1           NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
2           NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
3           NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
4           NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
5           NaN     NaN     4.0     NaN     NaN     NaN     NaN     NaN   
...         ...     ...     ...     ...     ...     ...     ...     ...   
667         NaN     NaN     NaN     NaN     NaN     4.0     NaN     NaN   
668         NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
669         NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
670         4.0     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
671         5.0     NaN     NaN     NaN     NaN     NaN     NaN     NaN   

movieId  9       10     

In [67]:
movies=pd.read_csv("tmdb_5000_movies.csv")
movies.rename(columns={'id':'movieId'}, inplace=True)
movies

,budget,genres,homepage,movieId,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4798,220000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",NaN,9367,"[{""id"": 5616, ""name"": ""united states\u2013mexi...",es,El Mariachi,El Mariachi just wants to play his guitar and ...,14.269792,"[{""name"": ""Columbia Pictures"", ""id"": 5}]","[{""iso_3166_1"": ""MX"", ""name"": ""Mexico""}, {""iso...",1992-09-04,2040920,81.0,"[{""iso_639_1"": ""es"", ""name"": ""Espa\u00f1ol""}]",Released,"He didn't come looking for trouble, but troubl...",El Mariachi,6.6,238
4799,9000,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 10749, ""...",NaN,72766,[],en,Newlyweds,A newlywed couple's honeymoon is upended by th...,0.642552,[],[],2011-12-26,0,85.0,[],Released,A newlywed couple's honeymoon is upended by th...,Newlyweds,5.9,5
4800,0,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 18, ""nam...",http://www.hallmarkchannel.com/signedsealeddel...,231617,"[{""id"": 248, ""name"": ""date""}, {""id"": 699, ""nam...",en,"Signed, Sealed, Delivered","""Signed, Sealed, Delivered"" introduces a dedic...",1.444476,"[{""name"": ""Front Street Pictures"

In [68]:
ratings=pd.read_csv("ratings_small.csv")
ratings.columns

Index(['userId', 'movieId', 'rating', 'timestamp'], dtype='object')

In [69]:
movies.columns

Index(['budget', 'genres', 'homepage', 'movieId', 'keywords',
       'original_language', 'original_title', 'overview', 'popularity',
       'production_companies', 'production_countries', 'release_date',
       'revenue', 'runtime', 'spoken_languages', 'status', 'tagline', 'title',
       'vote_average', 'vote_count'],
      dtype='object')

In [73]:
ratings_movies=pd.merge(ratings, movies, on="movieId")
ratings_movies[ratings_movies['userId']==2]
ratings_movies.shape

(18571, 23)

In [76]:
data=ratings_movies.pivot_table('rating', index='userId', columns='title').fillna(0)
data

(670, 856)

In [77]:
data=data.transpose()
data

userId,1,2,3,4,5,6,7,8,9,10,...,662,663,664,665,666,667,668,669,670,671
title,,,,,,,,,,,,,,,,,,,,,
10 Things I Hate About You,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0
12 Angry Men,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1408,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
15 Minutes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
16 Blocks,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"You, Me and Dupree",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Young Frankenstein,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0
Zodiac,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [80]:
movie_sim=cosine_similarity(data, data)

In [84]:
data.index

Index(['10 Things I Hate About You', '12 Angry Men', '1408', '15 Minutes',
       '16 Blocks', '20,000 Leagues Under the Sea', '2001: A Space Odyssey',
       '2046', '21 Grams', '25th Hour',
       ...
       'Willy Wonka & the Chocolate Factory', 'World Trade Center',
       'X-Men Origins: Wolverine', 'Y Tu Mamá También', 'You Only Live Twice',
       'You, Me and Dupree', 'Young Frankenstein', 'Zodiac', 'eXistenZ',
       'xXx'],
      dtype='object', name='title', length=856)

In [87]:
type(movie_sim)
movie_sim_df=pd.DataFrame(movie_sim, index=data.index, columns=data.index)
movie_sim_df

title,10 Things I Hate About You,12 Angry Men,1408,15 Minutes,16 Blocks,"20,000 Leagues Under the Sea",2001: A Space Odyssey,2046,21 Grams,25th Hour,...,Willy Wonka & the Chocolate Factory,World Trade Center,X-Men Origins: Wolverine,Y Tu Mamá También,You Only Live Twice,"You, Me and Dupree",Young Frankenstein,Zodiac,eXistenZ,xXx
title,,,,,,,,,,,,,,,,,,,,,
10 Things I Hate About You,1.000000,0.0,0.000000,0.182153,0.0,0.022069,0.085323,0.0,0.00000,0.103490,...,0.059856,0.0,0.161801,0.088076,0.0,0.0,0.097588,0.000000,0.000000,0.014121
12 Angry Men,0.000000,1.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.00000,0.000000,...,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000
1408,0.000000,0.0,1.000000,0.447214,0.0,0.173381,0.028245,0.0,0.00000,0.000000,...,0.146955,0.0,0.148968,0.140265,0.0,0.0,0.191675,0.000000,0.000000,0.000000
15 Minutes,0.182153,0.0,0.447214,1.000000,0.0,0.077538,0.050526,0.0,0.00000,0.129863,...,0.197160,0.0,0.216516,0.141138,0.0,0.0,0.085720,0.115684,0.121365,0.000000
16 Blocks,0.000000,0.0,0.000000,0.000000,1.0,0.000000,0.000000,0.0,0.00000,0.000000,...,0.000000,0.0,0.130347,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"You, Me and Dupree",0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.00000,0.000000,...,0.000000,0.0,0.000000,0.000000,0.0,1.0,0.000000,0.000000,0.000000,0.000000
Young Frankenstein,0.097588,0.0,0.191675,0.085720,0.0,0.204590,0.115720,0.0,0.00000,0.000000,...,0.230622,0.0,0.423840,0.236086,0.0,0.0,1.000000,0.214856,0.110536,0.204346
Zodiac,0.000000,0.0,0.000000,0.115684,0.0,0.014016,0.222842,0.0,0.00000,0.075115,...,0.359021,0.0,0.288208,0.201826,0.0,0.0,0.214856,1.000000,0.163801,0.105379


In [91]:
movie_sim_df['X-Men Origins: Wolverine'].sort_values(ascending=False)[1:11]

title
Romeo Must Die                        0.649625
The Wedding Planner                   0.631669
Dogtown and Z-Boys                    0.501189
An Unfinished Life                    0.485643
Conquest of the Planet of the Apes    0.474626
Reign Over Me                         0.458155
The Terminal                          0.445337
Young Frankenstein                    0.423840
Whale Rider                           0.394136
Madagascar                            0.392497
Name: X-Men Origins: Wolverine, dtype: float64

In [ ]:
rating.csv와 movies.csv 두 파일을 읽어서 추천시스템 제작
1. 아이텐 기반 추천
2. 사용자 기반 추천
